## Feedback

1.a) 1/1

1.b) 2/2

2.) 1.5/2
While your solution is good, it didn't include the requested common table expression. With that, it'd look something like this:

```sql
WITH financials AS (
    SELECT transaction_id::text AS id, total_paid AS amount, '+' AS type 
    FROM purchases 
    WHERE transaction_date BETWEEN '2022-09-15' AND '2022-09-28'
    UNION ALL
    SELECT personel_id::text AS id, salary / 2 AS amount, '-' AS type 
    FROM employee)
SELECT 
    SUM(CASE WHEN type = '+' THEN amount ELSE 0 END) AS total_income,
    SUM(CASE WHEN type = '-' THEN amount ELSE 0 END) AS total_expenses,
    SUM(CASE WHEN type = '+' THEN amount ELSE -amount END) AS profit
FROM financials;
```

3.a) 1/1

3.b) 2/2

---
---

# Homework 2
## Balázs Menkó (O67UT7)

In [1]:
# packages
import psycopg2
import psycopg2.extras
import pandas as pd

In [2]:
# Create connection
pgsql_settings = {
    'pguser' : 'menkobalazs1',
    'pgpasswd' : 'O67UT7',
    'pghost' : 'postgres-datasci.db-test',
    'pgport' : 5432,
    'pgdb' : 'menkobalazs1_homework',
    'schema' : 'public'
}

def connect_from_settings(settings):
    return psycopg2.connect(
        host = settings['pghost'],
        port = settings['pgport'],
        database = settings['pgdb'],
        user = settings['pguser'],
        password = settings['pgpasswd'],
        options=f'--search_path={settings["schema"]}'
    )

def run_query(query):
    "Run a query in one line"
    connection = connect_from_settings(pgsql_settings)
    cursor = connection.cursor(cursor_factory = psycopg2.extras.DictCursor)
    # Run the query
    cursor.execute(query)
    # Get the results
    dict_res = cursor.fetchall()
    # Since the results are in a list, we'll only ask for the keys() of the first element, 
    # and list the iterator returned by keys()
    df = pd.DataFrame(dict_res,columns=list(dict_res[0].keys())) 
    # Close everything
    cursor.close()
    connection.close()
    return df

# Create tables

```python
connection = connect_from_settings(pgsql_settings)
cursor = connection.cursor()
# Run the query
cursor.execute("""
    -- create_a_table: copy the generator text and run the code
""")
# COMMIT CHANGES
connection.commit()
# Close everything
cursor.close()
connection.close()
```

### Tables and columns:
- `branch`: location_id, location, manager, café, drive_thru, wi_fi_password
- `employee`: personel_id, first_name, last_name, date_of_birth, role, salary, supervisor_id, branch_id
- `purchases`: transaction_id, transaction_date, location_id, cashier_id, total_paid, items_sold
- `supplier`: branch_id, supplier_name, product
- `menu`: item_id, item_name, item_price, item_type

# Task 1
List all the items of expenses (-) and incomes (+) of the company for the two weeks from 2022.09.15-2022.09.28.! Note, that the employee salaries are given for a month so divide them by two! Along each entry display the personel_id if it belongs to an employee or the transaction_id if it is from a transaction.
- Give the query plan using of Relational Algebra operations! (Either tree or equation form is fine.) (1 points)
- Present the corresponding SQL query! (2 points)

#### Relational Algebra operations (equation form)

- Step 1: Income (Transactions)
$$ \sigma_{\text{transaction_date}\geq '2022-09-15'\, \land \,
\text{transaction_date}\leq '2022-09-28'}(\text{purchases}) $$

- Step 2: Expense (Salaries)
$$ \tau_{\text{salary}/2}(\pi_{\text{personel_id, salary}}(\text{employee})) $$

- Step 3: Union (Combine Incomes and Expenses)
$$ \tau_{\text{salary}/2}(\pi_{\text{personel_id, salary}}(\text{employee})) \cup
\pi_{\text{transaction_id, total_paid}}(\text{purchases})$$


In [3]:
# To change the data type, use the syntax *variable_name*::*new_type*
query = """
    SELECT transaction_id::text AS id, total_paid AS amount, '+' AS type 
    FROM purchases 
    WHERE transaction_date BETWEEN '2022-09-15' AND '2022-09-28'

        UNION ALL

    SELECT personel_id::text AS id, salary / 2 AS amount, '-' AS type 
    FROM employee
"""
run_query(query)

,id,amount,type
0,7d2c076803fe468882b88a7b695dbf4f,10.0,+
1,d25f6d2a790f4c37a1245e10a2df45f3,9.0,+
2,cd7b18d66bd649758266be6902df62c3,6.0,+
3,50e32d24c68144aead338e24cec306a9,5.0,+
4,1a9de3692b0440de947e99eefc1a77f5,4.0,+
...,...,...,...
1025,129,75.0,-
1026,220,63.5,-
1027,304,74.0,-
1028,402,75.0,-


# Task 2
Calculate how much profit did the company make in that two weeks using the result from the previous exercise as a CTE (Common Table Expression) in an SQL query! (2 poins)

In [4]:
query = """
    SELECT 
        SUM(CASE WHEN type = '+' THEN amount ELSE 0 END) AS total_income,
        SUM(CASE WHEN type = '-' THEN amount ELSE 0 END) AS total_expenses,
        SUM(CASE WHEN type = '+' THEN amount ELSE -amount END) AS profit
    FROM (
        SELECT transaction_id::text AS id, total_paid AS amount, '+' AS type 
        FROM purchases 
        WHERE transaction_date BETWEEN '2022-09-15' AND '2022-09-28'

            UNION ALL

        SELECT personel_id::text AS id, salary / 2 AS amount, '-' AS type 
        FROM employee
    ) AS financials
"""
run_query(query)

,total_income,total_expenses,profit
0,7318.0,1492.0,5826.0


# Task 3
How many purchases were at each location, where the amount paid was less, than the average?\
Hint: Use Inner Join
- Give the query plan using of Relational Algebra operations! (Either tree or equation form is fine.) (1 points)
- Present the corresponding SQL query! (2 points)

#### Relational Algebra operations (equation form)

- Step 1: Calculate the average paid amount
$$ \gamma_{\text{avg(total_paid)}}(\text{purchases}) $$

- Step 2: Filter purchases below the average
$$ \sigma_{\text{total_paid}<\text{avg_total_paid}}(\text{purchases}) $$

- Step 3: Join purchases with branch locations
$$  \sigma_{\text{total_paid}<\text{avg_total_paid}}(\text{purchases}) 
\bowtie_{\text{purchases.location_id} =\text{branch.location_id}}\text{branch}  $$

- Step 4: Group and count purchases by location
$$  \gamma_{\text{location, count(*)}}
(\sigma_{\text{total_paid}<\text{avg_total_paid}}(\text{purchases}) 
\bowtie\text{branch}) $$

In [5]:
query = """
    SELECT branch.location, COUNT(*) AS purchase_count
    FROM purchases 
    INNER JOIN branch ON purchases.location_id = branch.location_id
    CROSS JOIN (
        SELECT AVG(total_paid) AS avg_total_paid
        FROM purchases
    ) AS avg_payment 
    WHERE purchases.total_paid < avg_payment.avg_total_paid
    GROUP BY branch.location
    ORDER BY location
"""
run_query(query)

,location,purchase_count
0,Cape Town,98
1,London,101
2,New York,108
3,Tokyo,105
4,Wellington,100
